1. Solve the following problems:
    1. $x^3 - 6x^2+11x -6 = 0$

    2. $x = cos(x)$ 

    3. $x_2 = x_1^2-x_1$, $\frac{x_1^2}{16} + x_2^2 =1$ 

    4. $e^{-e^{x_1+x_2}}=x_2(1+x_1^2)$, $x_1\cos(x_2)+x_2\sin(x_1)=\frac{1}{2}$

Problem 1: Roots = [1.0, 2.0, 3.0]  
Problem 2: solution 0.7390851332151607  
Problem 3: Solutions (x1, x2) = [1.58100555, 0.91857299], [-0.61274319,  0.98819741]  
Problem 4: Solution (x1, x2) = [0.44448862, 0.13907865]  

In [18]:
import numpy as np
import scipy as sp

# 1
def f1(x): 
    y1 = x**3 - 6*x**2 + 11*x - 6
    return y1

intervals = [(0.5, 1.5), (1.5, 2.5), (2.5, 3.5)]  # Divide the range into subintervals
roots = []

for x_a, x_b in intervals:
    try:
        root = sp.optimize.bisect(f1, x_a, x_b)
        roots.append(root)
    except ValueError:
        pass  # Skip subintervals where no root is found

print("Problem 1: Roots =", roots)

# 2
def g2(x): 
    y2 = np.cos(x)
    return y2

x_init = 0.5  
x_fixed = sp.optimize.fixed_point(g2, x_init)
print("Problem 2: Fixed point at x =", x_fixed)

# Problem 3: Solve system of equations x2 = x1^2 - x1 and (x1^2)/16 + x2^2 = 1
def g3_system(x):
    x1, x2 = x
    eq1 = x2 - (x1**2 - x1)  # From x2 = x1^2 - x1
    eq2 = (x1**2)/16 + x2**2 - 1  # From (x1^2)/16 + x2^2 = 1
    return [eq1, eq2]

initial_guesses = [[0.5, 0.5], [-0.5, 0.5]]   # Multiple initial guesses
solutions = []

for guess in initial_guesses:
    try:
        sol = sp.optimize.root(g3_system, guess).x
        solutions.append(sol)
    except ValueError:
        pass  # Skip guesses where no solution is found

print("Problem 3: Solutions (x1, x2) =", solutions)


# Problem 4: Solve the system of equations:
# e^(-e^(x1 + x2)) = x2(1 + x1^2)
# x1*cos(x2) + x2*sin(x1) = 1/2
def f4_system(x):
    x1, x2 = x
    eq1 = np.exp(-np.exp(x1 + x2)) - x2 * (1 + x1**2)
    eq2 = x1 * np.cos(x2) + x2 * np.sin(x1) - 0.5
    return [eq1, eq2]

x_init = [0.1, 0.1]  # Initial guess for [x1, x2]
x_solution = sp.optimize.root(f4_system, x_init).x
print("Problem 4: Solution (x1, x2) =", x_solution)

Problem 1: Roots = [1.0, 2.0, 3.0]
Problem 2: Fixed point at x = 0.7390851332151607
Problem 3: Solutions (x1, x2) = [array([1.58100555, 0.91857299]), array([-0.61274319,  0.98819741])]
Problem 4: Solution (x1, x2) = [0.44448862 0.13907865]


2. Question 3.7 in the textbook.

Consider a simple endowment economy with three agents and two goods.  Agent $i$ is initially endowed with $e_{ij}$ units of good $j$ and maximizes utility
$$ U_i(x) = \sum_{j=1}^2 a_{ij} (v_{ij}+1)^{-1} x_{ij}^{v_{ij}+1},$$
subject to the budget constraint
$$ \sum_{j=1}^2 p_j x_{ij} = \sum_{j=1}^2 p_j e_{ij}. $$
Here, $x_{ij}$ is the amount of good $j$ consumed by agent $i$, $p_j$ is the market price of good $j$, and $a_{ij}>0$ and $v_{ij}<0$ are preference parameters.

A competitive general equilibrium for the endowment economy is a pair of relative prices, $p_1$ and $p_2$, normalized to sum to one, such that all the goods markets clear and each agent maximizes utility subject to his budget constraints.  Compute the competitive general equilibrium for the following parameters:

|(i,j)   | a_{ij}   | v_{ij}   | e_{ij} |
|--------|----------|----------|----------|
|(1,1)   | 2.0      | -2.0     |2.0       |
|(1,2)   |1.5       |-0.5      |3.0       |
|(2,1)   |1.5       |-1.5      |1.0       |
|(2,2)   |2.0       |-0.5      |2.0       |
|(3,1)   |1.5       |-0.5      |4.0       |
|(3,2)   |2.0       |-1.5      |0.0       |


Equilibrium prices (p1, p2): [0.20851254 0.79148746]  
Agent 1 equilibrium consumption: [2.90042859 2.76278759]  
Agent 2 equilibrium consumption: [2.36855071 1.63946368]  
Agent 3 equilibrium consumption: [1.7310207  0.59774873]

In [2]:
import numpy as np
from scipy.optimize import fsolve, root_scalar

# Data arrays: a[i, j], v[i, j], e[i, j]
a = np.array([[2.0, 1.5],
              [1.5, 2.0],
              [1.5, 2.0]])
v = np.array([[-2.0, -0.5],
              [-1.5, -0.5],
              [-0.5, -1.5]])
e = np.array([[2.0, 3.0],
              [1.0, 2.0],
              [4.0, 0.0]])

def agent_demand(lambda_val, p, a_i, v_i):
    """
    Returns the consumption vector x for an agent given lambda, prices, a_i and v_i.
    p, a_i, v_i are arrays of length 2.
    """
    # We compute x_j = ((lambda * p_j) / a_i[j])^(1/v_i[j])
    # Because v_i[j] is negative, (1/v_i[j]) is negative.
    return ((lambda_val * p) / a_i) ** (1.0 / v_i)

def lambda_equation(lambda_val, p, a_i, v_i, income):
    """
    For a given lambda, returns the excess (budget spending - income).
    We need to find lambda such that this equals 0.
    """
    x = agent_demand(lambda_val, p, a_i, v_i)
    return np.dot(p, x) - income

def solve_agent_lambda(p, a_i, v_i, income):
    """
    Solves for lambda for a given agent.
    """
    # Use an initial guess.
    lambda_guess = 1.0
    lambda_sol, = fsolve(lambda L: lambda_equation(L, p, a_i, v_i, income), lambda_guess)
    return lambda_sol

def agent_consumption(p, a_i, v_i, e_i):
    """
    Given prices p (an array of length 2), parameters a_i, v_i, and endowment e_i for agent i,
    compute the optimal consumption vector.
    """
    income = np.dot(p, e_i)
    lam = solve_agent_lambda(p, a_i, v_i, income)
    x = agent_demand(lam, p, a_i, v_i)
    return x

# ---------------------------
# Market clearing: excess demand for good 1
# ---------------------------
def excess_demand(p_val):
    """
    p_val is the candidate for p[0]. We set p = [p_val, 1-p_val].
    This function returns the excess demand in good 1 (good indexed 0).
    """
    p = np.array([p_val, 1.0 - p_val])
    total_demand_good1 = 0.0
    for i in range(3):
        x_i = agent_consumption(p, a[i, :], v[i, :], e[i, :])
        total_demand_good1 += x_i[0]
    total_supply_good1 = np.sum(e[:, 0])
    return total_demand_good1 - total_supply_good1

# ---------------------------
# Solve for the equilibrium relative price p[0] (with p[0] in (0,1))
# ---------------------------

sol = root_scalar(excess_demand, bracket=[0.01, 0.99], method='bisect')
if sol.converged:
    p0_eq = sol.root
    p_eq = np.array([p0_eq, 1.0 - p0_eq])
    print("Equilibrium prices (p1, p2):", p_eq)
else:
    print("Root finding did not converge for the equilibrium price.")

# ---------------------------
# Compute and print the equilibrium consumption for each agent
# ---------------------------
for i in range(3):
    x_i = agent_consumption(p_eq, a[i, :], v[i, :], e[i, :])
    print(f"Agent {i+1} equilibrium consumption:", x_i)


Equilibrium prices (p1, p2): [0.20851254 0.79148746]
Agent 1 equilibrium consumption: [2.90042859 2.76278759]
Agent 2 equilibrium consumption: [2.36855071 1.63946368]
Agent 3 equilibrium consumption: [1.7310207  0.59774873]


3. Question 3.8 in the textbook.

Potatoes are storable intraseasonaly, but not interseasonaly, implying that the harvest is entirely consumed over the subsequent twelve months, $i=1,2$. Denoting initial supply by $s$ and consumption in period $i$ by $c_i$, material balance requires that:
$$
s = \sum_{i=1}^{2}c_i.
$$
Competition among storers possessing perfect foresight eliminate arbitrage opportunities; thus,
$$
p_i + \kappa = \delta p_{i+1},\qquad i=1,
$$
where $p_i$ is equilibrium price in period $i$, $\kappa=0.2$ is per-period unit cost of storage, and $\delta=0.95$ is per-period discount factor.  Demand, assumed the same across periods, is given by
$$
p_i=c_i^{-5}.
$$
Compute the equilibrium period 1 and period 2 prices for $s=1$, $s=2$, and $s=2$.


S = 1, Period 1 Price: 31.0925, Period 2 Price: 32.9395  
S = 2, Period 1 Price: 0.8814, Period 2 Price: 1.1383  
S = 3, Period 1 Price: 0.0680, Period 2 Price: 0.2821  

In [34]:
import numpy as np
from scipy.optimize import fsolve

# Parameters
kappa = 0.2  # Per-period unit cost of storage
delta = 0.95  # Per-period discount factor

# Demand function
def demand(c):
    return c**-5

# Equations to solve for equilibrium
def equilibrium_system(variables, s):
    c1, c2 = variables
    p1 = demand(c1)
    p2 = demand(c2)

    eq1 = c1 + c2 - s  # Material balance
    eq2 = p1 + kappa - delta * p2  # Arbitrage condition

    return [eq1, eq2]

# Solve for different values of initial supply
results = {}
for s in [1, 2, 3]:
    initial_guess = [s / 2, s / 2]  # Initial guess for c1 and c2
    solution = fsolve(equilibrium_system, initial_guess, args=(s,))

    c1, c2 = solution
    p1 = demand(c1)
    p2 = demand(c2)

    results[s] = (p1, p2)

# Display results
for s, (p1, p2) in results.items():
    print(f"Supply: {s}, Period 1 Price: {p1:.4f}, Period 2 Price: {p2:.4f}")


Supply: 1, Period 1 Price: 31.0925, Period 2 Price: 32.9395
Supply: 2, Period 1 Price: 0.8814, Period 2 Price: 1.1383
Supply: 3, Period 1 Price: 0.0680, Period 2 Price: 0.2821


Homework #4


2. A consumer's preferences over the commodiies $x_1$, $x_2$, and $x_3$ are characterized by the Stone-Geary utility function
$$ U(x) =\sum_{i=1}^{3} \beta_i ln(x_i-\gamma_i)$$
The consumer wants to maximize his utility subject to the budtget constraint
$$ \sum_{i=1}^3 p_i x_i \le I $$
where $p_i>0$ denotes the prices of $x_i$, $I$ denotes income. Solve the problem numerically under the following parameterization: $\beta_1=0.1$, $\beta_2=0.2$, $\beta_3=0.3$, $\gamma_1=0.1$, $\gamma_2=0.2$, $\gamma_3=0.3$, $p_1=0.2$, $p_2=0.6$, $p_3=0.2$, and $I=2$.

$x_1$ = 1.5999999999999999  
$x_2$ = 1.2  
$x_3$ = 4.799999999999999  

In [ ]:
from scipy.optimize import fsolve

# Parameters
beta = [0.1, 0.2, 0.3]  # Preference weights
gamma = [0.1, 0.2, 0.3]  # Subsistence levels
p = [0.2, 0.6, 0.2]  # Prices
I = 2  # Income

# Budget constraint equation to solve for lambda
def budget_constraint(lambda_):
    # Compute the demand for each good based on lambda
    demands = [gamma[i] + beta[i] / (lambda_ * p[i]) for i in range(3)]
    # Check if the budget constraint holds
    return sum(p[i] * demands[i] for i in range(3)) - I

# Solve for lambda
lambda_solution = fsolve(budget_constraint, 1.0)[0]

# Compute the optimal consumption for each good
x_optimal = [gamma[i] + beta[i] / (lambda_solution * p[i]) for i in range(3)]

lambda_solution, x_optimal
